<a href="https://colab.research.google.com/github/MaryamNourii/EmotionDetection/blob/Define_Train_Model/ED_Bert_fa_Armandataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.2 MB/s eta 0:00:00


In [2]:

from transformers import AutoConfig, AutoTokenizer, TFAutoModel,TFBertForSequenceClassification

import tensorflow as tf
from keras import layers
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight

from transformers import TFBertModel, BertTokenizerFast, BertConfig

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import TruncatedNormal

In [21]:
MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
# MODEL_NAME_OR_PATH = 'bert-base-uncased'

In [5]:
data = pd.read_csv('/content/CleanData_arman.csv', sep=',')

In [6]:
data[data['text'].isnull()]

,text,label


In [37]:
data['target'] = data['label'].map({'SAD':0, 'HATE':1, 'FEAR':2, 'ANGRY':3, 'HAPPY':4, 'SURPRISE':5, 'OTHER':6})
data_x = data['text']
data_y = data['target']

In [38]:
max_length = data_x.apply(lambda x: len(x.split())).max()
max_length  

48

In [39]:
config = BertConfig.from_pretrained(MODEL_NAME_OR_PATH, output_hidden_states=False)
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = MODEL_NAME_OR_PATH, config = config)
transformer_model = TFAutoModel.from_pretrained(MODEL_NAME_OR_PATH, config = config)

Some layers from the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [40]:
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
attention_mask = Input(shape=(max_length,), name='attention_mask', dtype='int32')
token_ids = Input(shape=(max_length,), name='token_ids', dtype='int32')
inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

bert_model = transformer_model(inputs)[1]
pooled_output = Dropout(config.hidden_dropout_prob, name='pooled_output')(bert_model, training=False)

emotion = Dense(units=7, activation="sigmoid", kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='emotion')(pooled_output)
outputs = emotion

model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel')
model.layers[1].trainable = False

In [41]:
model.summary()
     

Model: "BERT_MultiLabel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 48)]         0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 48)]         0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  TFBaseModelOutputWi  162841344   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 48,                                  

In [42]:
train_x,test_x,train_y,test_y = train_test_split(data_x,data_y,test_size=0.2)
train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.1)

In [43]:
train_token = tokenizer(
    text = train_x.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

val_token = tokenizer(
    text = val_x.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

test_token = tokenizer(
    text = test_x.to_list(),
    add_special_tokens = True,
    max_length = max_length,
    truncation = True,
    padding = 'max_length', 
    return_tensors = 'tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [44]:
train = {'input_ids': train_token['input_ids'], 'attention_mask': train_token['attention_mask'],'token_ids': train_token['token_type_ids']}
val = {'input_ids': val_token['input_ids'], 'attention_mask': val_token['attention_mask'],'token_ids': val_token['token_type_ids']}
test = {'input_ids': test_token['input_ids'], 'attention_mask': test_token['attention_mask'],'token_ids': test_token['token_type_ids']}

In [45]:
train_tensor = tf.data.Dataset.from_tensor_slices((train, train_y)).shuffle(len(train)).batch(16)
val_tensor = tf.data.Dataset.from_tensor_slices((val, val_y)).shuffle(len(val)).batch(16)
test_tensor = tf.data.Dataset.from_tensor_slices((test, test_y)).shuffle(len(test)).batch(16)

In [46]:
optimizer = Adam(
    learning_rate=5.e-05,
    )

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = [metric])


log_dir='tb_bert'
model_save_path='bert_model.h5'
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,
                                                save_weights_only=True,
                                                monitor='val_loss',mode='min'
                                                ,save_best_only=True),
                                                tf.keras.callbacks.TensorBoard(log_dir=log_dir)]

history = model.fit(train_tensor, 
                    epochs=4, 
                    validation_data=val_tensor,
                    callbacks=callbacks)

Epoch 1/4


/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:566: UserWarning: Input dict contained keys ['token_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


293/293 [==============================] - 89s 231ms/step - loss: 1.4035 - accuracy: 0.4795 - val_loss: 1.1754 - val_accuracy: 0.5988
Epoch 2/4
293/293 [==============================] - 63s 213ms/step - loss: 0.9035 - accuracy: 0.6908 - val_loss: 1.2689 - val_accuracy: 0.5854
Epoch 3/4
293/293 [==============================] - 63s 213ms/step - loss: 0.5539 - accuracy: 0.8220 - val_loss: 1.3842 - val_accuracy: 0.5758
Epoch 4/4
293/293 [==============================] - 62s 212ms/step - loss: 0.3589 - accuracy: 0.8889 - val_loss: 1.5866 - val_accuracy: 0.5969


In [47]:
y_pred_proba = model.predict(test_tensor)

82/82 [==============================] - 8s 65ms/step


In [ ]:
y_pred_proba

array([[0.992389  , 0.20858517, 0.22776924, ..., 0.95903873, 0.06093171,
        0.03556877],
       [0.07805462, 0.9997267 , 0.33255666, ..., 0.10958179, 0.14960788,
        0.07282835],
       [0.9991611 , 0.49396893, 0.12589207, ..., 0.41559   , 0.10181875,
        0.02942073],
       ...,
       [0.21426617, 0.16826105, 0.09782495, ..., 0.98526716, 0.97582895,
        0.03812208],
       [0.09479297, 0.99965847, 0.21344616, ..., 0.1591344 , 0.16077018,
        0.09186505],
       [0.361214  , 0.15158099, 0.11224746, ..., 0.94728696, 0.239922  ,
        0.03555888]], dtype=float32)